In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

In [2]:
crl = ["darkblue","slategray","slategray","crimson","crimson"]

background = "#ffffff"

In [3]:

# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key

In [4]:
def get_top_imports(hs):

    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    end_use = "hs?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC"

    surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

    surl  = surl + my_key + "&time=" + "from+2024-01" + "&COMM_LVL=" + hs 

    url = surl + "&CTY_CODE=" + "2010"

    # Mexico is 2010
    # Canada is 1220
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")

    df["imports"] = df["CON_VAL_MO"].astype(float)
    
    df["duty"] = df["CAL_DUT_MO"].astype(float)
        
    df[hs] = df["I_COMMODITY"].astype(str)

    df.drop(["CON_VAL_MO", "I_COMMODITY", "COMM_LVL"], axis = 1, inplace = True)
    
    df["share"] = df.imports / df.imports.sum()
    
    df["tariff"] = df["duty"] / df["imports"]
    
    grp = df.groupby([hs])

    top_products = grp.agg({"imports":"sum", "I_COMMODITY_SDESC":"first", "share": "sum", "tariff":"mean"})
       
    top_products["hs-code"] = top_products.index

    top_products["name"] = hs + " " + top_products["hs-code"] + ": " + top_products["I_COMMODITY_SDESC"].str[0:30]

    top_products["color"] = "#006847"
    ##ff0000
    
    return top_products.sort_values(by = ["imports"], ascending = False)[0:20]

In [5]:
foo = get_top_imports("HS2")

<Response [200]>


In [6]:
foo

,imports,I_COMMODITY_SDESC,share,tariff,hs-code,name,color
HS2,,,,,,,
87,1.354147e+11,"VEHICLES, EXCEPT RAILWAY OR TRAMWAY, AND PARTS...",0.268783,3.462838e-03,87,"HS2 87: VEHICLES, EXCEPT RAILWAY OR TR",#ff0000
84,1.056129e+11,"NUCLEAR REACTORS, BOILERS, MACHINERY ETC.; PARTS",0.209630,1.826892e-03,84,"HS2 84: NUCLEAR REACTORS, BOILERS, MAC",#ff0000
85,8.692565e+10,ELECTRIC MACHINERY ETC; SOUND EQUIP; TV EQUIP;...,0.172538,2.782822e-03,85,HS2 85: ELECTRIC MACHINERY ETC; SOUND,#ff0000
90,2.289012e+10,"OPTIC, PHOTO ETC, MEDIC OR SURGICAL INSTRMENTS...",0.045434,6.597610e-04,90,"HS2 90: OPTIC, PHOTO ETC, MEDIC OR SUR",#ff0000
27,1.547473e+10,"MINERAL FUEL, OIL ETC.; BITUMIN SUBST; MINERAL...",0.030716,5.371319e-04,27,"HS2 27: MINERAL FUEL, OIL ETC.; BITUMI",#ff0000
22,1.288282e+10,"BEVERAGES, SPIRITS AND VINEGAR",0.025571,2.256567e-06,22,"HS2 22: BEVERAGES, SPIRITS AND VINEGAR",#ff0000
94,1.269509e+10,"FURNIT;BEDDING,MATTRES;LUMINAIRES,LIGHT FIX;PR...",0.025198,7.097103e-04,94,"HS2 94: FURNIT;BEDDING,MATTRES;LUMINAI",#ff0000
08,1.076101e+10,EDIBLE FRUIT & NUTS; CITRUS FRUIT OR MELON PEEL,0.021359,4.763228e-05,08,HS2 08: EDIBLE FRUIT & NUTS; CITRUS FR,#ff0000
07,9.913014e+09,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,0.019676,1.309915e-05,07,HS2 07: EDIBLE VEGETABLES & CERTAIN RO,#ff0000


In [7]:
def make_source(df):
    
    df["position"] = df.reset_index().index.values
        
    df["hover_label"] = (df["imports"]/1000000000).map('{:,.1f}'.format)
    
    df["hover_label_2"] = (df["share"]*100).map('{:,.1f}'.format)
    
    df["hover_label_3"] = (df["tariff"]*100).map('{:,.1f}'.format)
    
    source = ColumnDataSource(df)
    
    return source

In [8]:
def make_bar_chart(df, hs):

    width = 600
    height = 500

    source = make_source(df)
        
    p = figure(plot_height=height, plot_width = width, title= "Top US Imports from Mexico at " + hs+ "-level",
           toolbar_location = 'below',
           tools = "reset")
        
    p.vbar(x = "position", top = "imports", width = 0.6, alpha = 0.65,
       hatch_pattern = " ",hatch_alpha = 0.10, color = "color",
       source = source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)

##########################################################################
    TIMETOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95; border: 0px 0px 0px 0px">
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">@name</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Imports: $@hover_label Billion</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Share of Total: @hover_label_2%</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Applied Tariff: @hover_label_3%</span>
        </div>
    </div>
    """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    #p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '13pt'
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels

    p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
    p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    
    p.y_range.end = df.imports.max() + 0.10*df.imports.max()
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)
    return p

In [9]:
hs = "HS2"

foo = get_top_imports(hs)

p1 = make_bar_chart(foo, hs)

hs = "HS4"

foo = get_top_imports(hs)

p2 = make_bar_chart(foo, hs)

hs = "HS6"

foo = get_top_imports(hs)

p3 = make_bar_chart(foo, hs)

<Response [200]>
<Response [200]>
<Response [200]>


In [10]:

tab1 = Panel(child= p1, title="HS2")

tab2 = Panel(child= p2, title="HS4")

tab3 = Panel(child= p3, title="HS6")

output_file('.\\docs\\' + "us-imports-mexico.html")

div0 = Div(text = """Each bar represents the total sum of U.S. imports from Mexico in 2024 for a HS category. 
The top 20 import categories are displayed. Hover your cursor over each bar to learn more.
""", max_width=600, background = background )

div0.sizing_mode= "scale_both"
        
outfig = column(Tabs(tabs=[tab1,  tab2, tab3], tabs_location = "above"), div0, sizing_mode="scale_both")

show(outfig)